Team Members: Chaitanya Muppala(UIN:), Supriya RP, Vanisa A

In [1]:
import re
import concurrent.futures
from _collections import OrderedDict
from itertools import chain
from operator import itemgetter 
import pandas as pd

In [2]:
with open("Pride_and_Prejudice.txt", "r") as file:
    txtData = file.read()

In [3]:
def cleanData(file):
    file_cleaned = re.sub(r'\d+', '', txtData)
    x = file_cleaned.lower()
    x2 = x.splitlines()

    blank = []
    for line in x2:  
        if len(line) > 0: 
            linesp = line.split()                                 #split the line into sublist of words(each line)
            temp = []
            for word in linesp:                                   #iterate the word in each line
              temp.append("".join(re.findall("[a-zA-Z]+", word))) #extract only charactors, append words into sublist 
            blank.append(temp)
    return blank


In [4]:
def splitdata(data):
    
  #merge sublist into list
  split1 = []
  for i in data[0:5000]: 
    split1 += i
    
  #merge sublist into list
  split2 = []
  for i in data[5000:]: 
    split2 += i

  return split1,split2

In [5]:
# mapper function
def mapperFunc(input_data):
    # Generating [key,value = 1] pairs
    output_data = [[x, 1] for x in input_data]
    return output_data

In [6]:

def sortfunc(sortdata_in): 
 sortdata_in.sort(key = lambda x: x[0]) 
 return sortdata_in


In [7]:
## Partition two from a-m and n-z words
check1 = tuple('abcdefghijklm')
check2 = tuple('nopqrstuvwxyz')

def partition(check):  
  # using startswith()  
      res = [idx for idx in sortdata_out if idx[0][0].startswith(check)] 
      return res


In [8]:
# Reducer
def reducer(partitiondata):
    reducedwords = []
    count = 1
    for i in range(0,len(partitiondata)):
        if i < len(partitiondata)-1:
              if partitiondata[i] == partitiondata[i+1]:
                    count = count+1                              
        else :
            reducedwords.append([partitiondata[i][0],count])  
            count = 1 
    else: reducedwords.append(partitiondata[i])             
    return reducedwords


In [9]:
###KEEP####

import concurrent.futures
import logging
import queue
import random
import threading
import time

def producer(queue, event):
    """Pretend we're getting a number from the network."""
    while not event.is_set():
        message = random.randint(1, 101)
        logging.info("Producer got message: %s", message)
        queue.put(message)

    logging.info("Producer received event. Exiting")

def consumer(queue, event):
    """Pretend we're saving a number in the database."""
    while not event.is_set() or not queue.empty():
        message = queue.get()
        logging.info(
            "Consumer storing message: %s (size=%d)", message, queue.qsize()
        )

    logging.info("Consumer received event. Exiting")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    pipeline = queue.Queue(maxsize=10)
    # calling functions
    text_final = cleanData(txtData)

    # splitting text file for 5000:rest
    split1, split2 = splitdata(text_final)


    event = threading.Event()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread1 = executor.submit(mapperFunc, split1)
        split1 = thread1.result()
        thread2 = executor.submit(mapperFunc, split2)
        split2 = thread2.result()
        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()
    #sorting the data
    sortdata_in = split1 + split2
    sortdata_out = sortfunc(sortdata_in)
    #print(sortdata_out)
    
    # Calling first partition function to get a-m words
    partition1 = partition(check1)
    #print(partition1)
    
    # Calling first partition function to get n-z words
    partition2 = partition(check2)
    #print(partition2)
    
    #Calling reducer function with multithreading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread3 = executor.submit(reducer, partition1)
        reducer1 = thread3.result()
        thread4 = executor.submit(reducer, partition2)
        reducer2 = thread4.result()
        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()
    #Combining Reducer Output
    reducer_out = reducer1 + reducer2

21:12:18: Main: about to set event
21:12:18: Main: about to set event


#Sort Function

In [10]:
reducer_out

[['my', 56], ['my', 1], ['young', 51], ['young', 1]]

In [ ]:
def main():
    
    #Clean the data
    text_final = cleanData(txtData)

    #Splitting text file for 5000:rest
    split1, split2 = splitdata(text_final)


    event = threading.Event()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread1 = executor.submit(mapperFunc, split1)
        split1 = thread1.result()
        thread2 = executor.submit(mapperFunc, split2)
        split2 = thread2.result()
        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()
    #sorting the data
    sortdata_in = split1 + split2
    sortdata_out = sortfunc(sortdata_in)
    #print(sortdata_out)
    
    # Calling first partition function to get a-m words
    partition1 = partition(check1)
    #print(partition1)
    
    # Calling first partition function to get n-z words
    partition2 = partition(check2)
    #print(partition2)
    
    #Calling reducer function with multithreading
    with concurrent.futures.ThreadPoolExecutor() as executor:
        thread3 = executor.submit(reducer, partition1)
        reducer1 = thread3.result()
        thread4 = executor.submit(reducer, partition2)
        reducer2 = thread4.result()
        time.sleep(0.1)
        logging.info("Main: about to set event")
        event.set()
    #Combining Reducer Output
    reducer_out = reducer1 + reducer2
    
    